# Bitcoin link prediction


In [1]:
import os
import sys

import json
from pprint import pprint

import requests
import requests_cache

from itertools import combinations_with_replacement as cwr

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import networkx as nx

from hiveplot import HivePlot
from gensim.models import Word2Vec

from sklearn.metrics import roc_auc_score as auc

#from node2vec import node2vec as n2v
from node2vec import node2vec as n2v

from importlib import reload

reload(n2v)

plt.style.use('ggplot')
%matplotlib inline

In [2]:
graph_df = pd.read_csv("../data/btc_df.csv")
# Filter out the mining transactions
graph_df = graph_df.loc[~(graph_df.addr_from == 'mining'), :]
graph_df = graph_df.rename(columns={'value': 'weight'})
print(graph_df.shape)
graph_df.head()

(58664, 5)


,weight,spent_flag,height,addr_from,addr_to
171,1000000000,True,170,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1Q2TWHE3GMdB6BZKafqwxXtWAWgFt5Jvm3
172,4000000000,True,170,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S
183,3000000000,True,181,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S
184,1000000000,False,181,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1DUDsfc23Dv9sPMEk5RsrtfzCw5ofi5sVW
186,100000000,True,182,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1LzBzVqEeuQyjD2mRWHes3dgWrT9titxvq


In [27]:
height_max = 40000
height_train = int(height_max*0.9)
graph_df = graph_df.loc[graph_df.height < height_max, :]
graph_train = graph_df.loc[graph_df.height < height_train, :]
graph_test = graph_df.loc[graph_df.height >= height_train, :]
graph_df.head()

,weight,spent_flag,height,addr_from,addr_to
171,1000000000,True,170,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1Q2TWHE3GMdB6BZKafqwxXtWAWgFt5Jvm3
172,4000000000,True,170,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S
183,3000000000,True,181,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S
184,1000000000,False,181,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1DUDsfc23Dv9sPMEk5RsrtfzCw5ofi5sVW
186,100000000,True,182,12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S,1LzBzVqEeuQyjD2mRWHes3dgWrT9titxvq


# Create the bitcoin transaction graph and extract key metrics

In [28]:
edge_df = graph_df[['addr_from', 'addr_to']]#, 'weight']]
edge_df['data'] = graph_df[['weight', 'height']].to_dict(orient='records')
#edge_df['data'] = graph_df[['weight']].to_dict(orient='records')
edge_list = edge_df.values.tolist()

edge_train = graph_train[['addr_from', 'addr_to']]#, 'weight']]
edge_train['data'] = graph_train[['weight', 'height']].to_dict(orient='records')
#edge_df['data'] = graph_df[['weight']].to_dict(orient='records')
edge_train_list = edge_train.values.tolist()

edge_test = graph_test[['addr_from', 'addr_to']]#, 'weight']]
edge_test['data'] = graph_test[['weight', 'height']].to_dict(orient='records')
#edge_df['data'] = graph_df[['weight']].to_dict(orient='records')
edge_test_list = edge_test.values.tolist()

/Users/pavel/miniconda3/envs/ra/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/pavel/miniconda3/envs/ra/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pavel/miniconda3/envs/ra/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [30]:
G = nx.MultiDiGraph(edge_list)#[0:5000])
G_train = nx.MultiDiGraph(edge_train_list)
G_test = nx.MultiDiGraph(edge_test_list)
#nx.draw_networkx(G, with_labels=False, node_size = 80)
print("Number of nodes: {0}\nNumber of edges: {1}".format(G.number_of_nodes(), G.number_of_edges()))
print("Number of train nodes: {0}\nNumber of edges: {1}".format(G_train.number_of_nodes(), G_train.number_of_edges()))
print("Number of test nodes: {0}\nNumber of edges: {1}".format(G_test.number_of_nodes(), G_test.number_of_edges()))

Number of nodes: 5056
Number of edges: 6690
Number of train nodes: 3220
Number of edges: 4207
Number of test nodes: 1839
Number of edges: 2483


In [31]:
# Keep only nodes with valency at least 3
nodes_ge3 = [n for n in G.nodes() if G.in_degree(n) + G.out_degree(n) >=3]
G = G.subgraph(nodes_ge3)
#degs_in = G.in_degree()
#print(degs_in, '\n')
#degs_out = G.out_degree()

nodes_train_ge3 = [n for n in G_train.nodes() if G_train.in_degree(n) + G_train.out_degree(n) >=3]
G_train = G_train.subgraph(nodes_train_ge3)
#degs_train_in = G_train.in_degree()
#print(degs_in, '\n')
#degs_train_out = G_train.out_degree()

nodes_test_ge3 = [n for n in G_test.nodes() if G_test.in_degree(n) + G_test.out_degree(n) >=3]
G_test = G_test.subgraph(nodes_test_ge3)


In [33]:
G_und = G.to_undirected()
G_train_und = G_train.to_undirected()
G_test_und = G_test.to_undirected()
# Assign transaction amount as weight
print("Number of nodes: {0}\nNumber of edges: {1}".format(G.number_of_nodes(), G.number_of_edges()))
print("Number of nodes: {0}\nNumber of edges: {1}".format(G_train.number_of_nodes(), G_train.number_of_edges()))
print("Number of nodes: {0}\nNumber of edges: {1}".format(G_test.number_of_nodes(), G_test.number_of_edges()))



Number of nodes: 136
Number of edges: 252
Number of nodes: 109
Number of edges: 208
Number of nodes: 28
Number of edges: 42


In [34]:
# pub_key = '1XPTgDRhN8RFnzniWCddobD9iKZatrvH4'
# # Look at edges containing this public key
# for e in G.edges():
#     #if '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S' in e:
#     if pub_key in e:
#         print(e)

# degrees = [{n: G.in_degree(n) + G.out_degree(n)} for n in G.nodes()]
# degrees_df = pd.DataFrame.from_dict(degrees)
# degrees_df

## Decorate nodes and edges

In [35]:
# nx.set_node_attributes(G, "degree_in", degs_in)
# nx.set_node_attributes(G, "degree_out", degs_out)
# Assign "pure_miner" flag

In [36]:
#for n, d in G.nodes(data=True):
#    print(n, d)

#nodes['bc_mine'] = [(n,d) for n, d in G.nodes(data=True) if n == "mining"]
#G.node['1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa']["pure_miner"] = True
#G.node['1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa']

# # Find transaction recipients
# sender_keys = frozenset(np.unique([pub_key for pub_key in graph_df['addr_from'] if pub_key != 'mining']))
# #sender_key_mask = graph_df['addr_from'].isin(sender_keys)
#miner_addr = frozenset(graph_df.loc[graph_df['addr_from'] == 'mining', 'addr_to'])
#receiver_keys = frozenset(graph_df.loc[graph_df['addr_from'] != 'mining', 'addr_to'])

# mining_edges = []
# for i in range(graph_df.shape[0]):
#     row = graph_df.iloc[i]
#     if row['addr_from'] == 'mining':
#         mining_edges.append((row['addr_from'], row['addr_to']))
        
# mining_train_edges = []
# for i in range(graph_train.shape[0]):
#     row = graph_train.iloc[i]
#     if row['addr_from'] == 'mining':
#         mining_train_edges.append((row['addr_from'], row['addr_to']))
#miners = miner_addr#.difference(sender_keys)
#player = sender_keys.difference(pure_miner)

# for n in G.nodes():
#     if n in miner_addr:
#         G.node[n]["miner"] = True
#     else:
#         G.node[n]["miner"] = False
# Identify nodes that have received mining btc but never sent btc
#sum(np.multiply(miner_mask, ~sender_key_mask))
#pure_miner_keys = graph_df.loc[~graph_df['addr_to'].isin(sender_keys), 'addr_from']

In [37]:
# Label self payments
# TODO: This only gets one edge per pair of nodes, not multi-edges
# for u,v,d in G.edges(data=True):
#     if u == v:
#         G.edge[u][v][0]['selfie'] = True
#     else:
#         G.edge[u][v][0]['selfie'] = False
#= [(u,v,d) for u,v,d in G.edges(data=True) if u == v]

In [38]:
# Remove mining transactions and create edge list for node2vec
#nodes_no_mine = [n for n in G.nodes() if G.node[n]['miner'] ]
#G = G.subgraph(nodes_no_mine)
#G.remove_edges_from(mining_edges)
#G_und = G.to_undirected()

#G_train.remove_edges_from(mining_train_edges)
# G_train_und = G_train.to_undirected()
# # Assign transaction amount as weight
# print("Number of nodes: {0}\nNumber of edges: {1}".format(G.number_of_nodes(), G.number_of_edges()))
# print("Number of nodes: {0}\nNumber of edges: {1}".format(G_train.number_of_nodes(), G_train.number_of_edges()))


In [39]:
# for n in G.nodes():
#     print(G[n])

In [40]:
def learn_embeddings(walks, dimensions, window_size, workers, n_iter=1):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    
    # Python 2 or 3
    if sys.version_info.major < 3:
        walks = [map(str, walk) for walk in walks]
    else: # Python 3
        walks = [list(map(str,walk)) for walk in walks]
        
    model = Word2Vec(walks, size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, iter=n_iter)
    #model.save_word2vec_format(args.output)
    return model

    


## Compute metrics for pairs of nodes

In [41]:

# edge_metrics = {}
# unique_edges = list(set(G_und.edges_iter()))
# #for e in unique_edges:
# for e in G_und.edges_iter():
#     edge_metrics[e] = {'common_neighbors': len(list(nx.common_neighbors(G_und, e[0], e[1]))),
#                       'hadamard': np.dot(model.wv[e[0]], model.wv[e[1]])}
# #   edge_metrics[str(e)]['hadamard'] = np.dot(model.wv[e[0]], model.wv[e[1]])

#nx.triadic_census(G)
#print(e, edge_metrics[e])

In [42]:
# Split into train and test by blockheight
# height_max = np.max([d['height'] for (u,v,d) in G.edges(data=True)])
# height_max = 60000
# train_test_split = int(0.98*height_max)
# print(train_test_split, height_max)
# e_train = [(u,v) for (u,v,d) in G.edges(data=True) if d['height'] < train_test_split]
# e_test = [(u,v) for (u,v,d) in G.edges(data=True) if d['height'] >= train_test_split]
# nodes_train = list(set([item for sublist in e_train for item in sublist]))
# nodes_test = list(set([item for sublist in e_test for item in sublist]))
# print(len(e_train), len(e_test))
# print(len(nodes_train), len(nodes_test))

In [43]:
reload(n2v)
p, q = 1,3
# G_n2v = n2v.Graph(G, True, p, q)
# G_n2v.preprocess_transition_probs()


# walks = G_n2v.simulate_walks(20, 10)
G_n2v = n2v.Graph(G_train, True, p, q)
G_n2v.preprocess_transition_probs()


walks = G_n2v.simulate_walks(20, 10)


#walks
long_walks = [w for w in walks if len(w) > 2]
#long_walks
dimensions = 100
window_size = 5
workers =4
n_iter = 10
#walks = longer_walks
#walks = [map(str, walk) for walk in walks]
#model = Word2Vec(walks, size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, iter=n_iter)
model = learn_embeddings(walks, dimensions, window_size, workers, n_iter=n_iter)

Walk iteration:
1 / 20
2 / 20
3 / 20
4 / 20
5 / 20
6 / 20
7 / 20
8 / 20
9 / 20
10 / 20
11 / 20
12 / 20
13 / 20
14 / 20
15 / 20
16 / 20
17 / 20
18 / 20
19 / 20
20 / 20


In [59]:
pairs_train = cwr(G_train.nodes(), 2)
pairs_train_list = list(pairs_train)
#node_pairs = cwr(nodes_train, 2)
#nodes_train = G_train.nodes()
pairs_test = cwr(G_train.nodes(), 2)
pairs_test_list = list(set(pairs_test).intersection(set(pairs_train_list)))

In [62]:
e_train = G_train.edges()
e_test = G_test.edges()

In [63]:
train = []
def common_neighbor(pair, G):
    com_nbr = len(list(nx.common_neighbors(G_und, pair[0], pair[1])))

    return com_nbr

def hadamard(pair, G):
    hadamard = np.dot(model.wv[pair[0]], model.wv[pair[1]])
    
    return hadamard

In [73]:
%time edge_mask = [pair in e_train or (pair[1], pair[0]) in e_train for pair in pairs_train_list]
train_df = pd.DataFrame({'node_pair': pairs_train_list})
train_df['edge_mask'] = edge_mask
%time train_df['com_nbr'] = [common_neighbor(pair, G_train_und) for pair in pairs_train_list]
%time train_df['hadamard'] = [np.dot(model.wv[pair[0]], model.wv[pair[1]]) for pair in pairs_train_list]


CPU times: user 104 ms, sys: 1.85 ms, total: 106 ms
Wall time: 107 ms
CPU times: user 24.1 ms, sys: 269 µs, total: 24.4 ms
Wall time: 24.4 ms
CPU times: user 29 ms, sys: 7.69 ms, total: 36.7 ms
Wall time: 30 ms


In [74]:
print(train_df.shape)
sum(train_df['edge_mask'])

(5995, 4)


110

In [75]:
print(auc(train_df['edge_mask'], train_df['com_nbr']))
print(auc(train_df['edge_mask'], train_df['hadamard']))

0.744958677686
0.748536340465


In [76]:
%time edge_mask = [pair in e_test or (pair[1], pair[0]) in e_test for pair in pairs_test_list]
test_df = pd.DataFrame({'node_pair': pairs_test_list})
test_df['edge_mask'] = edge_mask
%time test_df['com_nbr'] = [common_neighbor(pair, G_test_und) for pair in pairs_test_list]
%time test_df['hadamard'] = [np.dot(model.wv[pair[0]], model.wv[pair[1]]) for pair in pairs_test_list]

print(auc(test_df['edge_mask'], test_df['com_nbr']))
print(auc(test_df['edge_mask'], test_df['hadamard']))

CPU times: user 21.3 ms, sys: 2.03 ms, total: 23.4 ms
Wall time: 22 ms
CPU times: user 38.7 ms, sys: 427 µs, total: 39.1 ms
Wall time: 39.4 ms
CPU times: user 34.5 ms, sys: 7.63 ms, total: 42.1 ms
Wall time: 36.5 ms
1.0
0.994327660994


In [52]:
print(test_df.shape)
sum(test_df['edge_mask'])

1

In [ ]:
#e_test